<H1> Example on using the datasets for quantum characterization and control </H1>

In [ ]:
# preample
import zipfile
import pickle
import os
import numpy as np

In [ ]:
'''The datalist below lists each different dataset category.
Each category comprises two compressed files:
(a) one for the non-distorted examples; and
(b) another for the distorted examples.

Each dataset is stored in a folder corresponding to its category on Cloudstor.
There are 52 datasets in the QDataSet and 26 categories.


To run the code:
1. Download both the distorted and non-distorted datasets from Cloudstor
(assuming both are to be used);
2. To select a dataset for the model (e.g. G_1q_X), uncomment the dataset
in the dataset list below.'''

In [ ]:
datalist = [
'G_1q_X',
# 'G_1q_XY',
# 'G_1q_XY_XZ_N1N5',
# 'G_1q_XY_XZ_N1N6',
# 'G_1q_XY_XZ_N3N6',
# 'G_1q_X_Z_N1',
# 'G_1q_X_Z_N2',
# 'G_1q_X_Z_N3',
# 'G_1q_X_Z_N4',
# 'G_2q_IX-XI_IZ-ZI_N1-N6',
# 'G_2q_IX-XI-XX',
# 'G_2q_IX-XI-XX_IZ-ZI_N1-N5',
# 'G_2q_IX-XI-XX_IZ-ZI_N1-N5',
# 'S_1q_X',
# 'S_1q_XY',
# 'S_1q_XY_XZ_N1N5',
# 'S_1q_XY_XZ_N1N6',
# 'S_1q_XY_XZ_N3N6',
# 'S_1q_X_Z_N1',
# 'S_1q_X_Z_N2',
# 'S_1q_X_Z_N3',
# 'S_1q_X_Z_N4',
# 'S_2q_IX-XI_IZ-ZI_N1-N6',
# 'S_2q_IX-XI-XX',
# 'S_2q_IX-XI-XX_IZ-ZI_N1-N5',
# 'S_2q_IX-XI-XX_IZ-ZI_N1-N6',
]

In [ ]:
'''Create two strings, one for each of the undistorted and distorted datasets.'''
data1 = datalist[0]
data2 = data1 + '_D'

In [ ]:
'''Check strings created correction'''
print(data1)
print(data2)

G_1q_X
G_1q_X_D


In [ ]:
'''Set the working directory to the location of the datasets, example below.'''
os.chdir('/projects/QuantumDS/QDataSet/' + data1)

## Characterization of an open quantum system

In [ ]:
'''Define a function to extract the data from the dataset.'''

def load_dataset(dataset_name, num_training_ex, num_testing_ex):
    
    # initialize empt lists for storing the data
    data_input         = []
    data_target        = []
    
    # 1) unzip the dataset zipfile
    fzip               = zipfile.ZipFile("%s.zip"%dataset_name, mode='r')
    
    # loop over example files
    #########################################################
    for idx_ex in range(num_training_ex + num_testing_ex):
    
        # 2) extract the example file from the dataset 
        fname = "%s_ex_%d"%(dataset_name, idx_ex)
        fzip.extract( fname )
    
        # 3) load the example file
        f     = open(fname,  "rb")
        data  = pickle.load(f)
    
        # 4) extract the useful information from the example file:
        
        # For noise spectroscopy, we need to extract a set of control pulse and the 
        # correpsonding informationally-complete observables
        
        # add the pair of input and target to the corresponding lists
        data_input.append( data["pulses"][0:1, :, 0, :])
        data_target.append( data["expectations"] )
        
        # 5) close and delete the example file
        f.close()
        os.remove(fname)
    #########################################################
    # 5) close the dataset zipfile
    fzip.close()
    
    # 6) split the data into training and testing
    data_input            = np.concatenate(data_input, axis=0)
    data_target           = np.concatenate(data_target, axis=0)
    
    training_input        = data_input[0:num_training_ex, :]
    testing_input         = data_input[num_training_ex:num_training_ex+num_testing_ex, :]
    
    training_target       = data_target[0:num_training_ex, :]
    testing_target        = data_target[num_training_ex:num_training_ex+num_testing_ex, :]
    
    return training_input, training_target, testing_input, testing_target

In [ ]:
'''Define the dataset parameters'''
dataset_name    = data1#"G_1q_XY_XZ_N1N5" # dataset name
num_training_ex = 7                 # number of training examples
num_testing_ex  = 3                 # number of testing examples

In [ ]:
# load the dataset
training_input, training_target, testing_input, testing_target = load_dataset(dataset_name, num_training_ex, num_testing_ex)

In [ ]:
'''Inspect the shape of the training and test dataframes (if need be).'''
print(training_input.shape, training_target.shape, testing_input.shape, testing_target.shape)

(7, 1024, 1) (7, 18) (3, 1024, 1) (3, 18)


In [ ]:
'''Set input parameter shape based on number of axes along which controls / noise is applied.'''
axnum = training_input.shape[2]
axnum2 = training_target.shape[1]
print(axnum, axnum2)

1 18


In [ ]:
# use the dataset for training and testing an ML algorithm

#############################################
##          PUT YOUR CODE HERE             ##

# trained_model  = my_training_function(training_input, training_target)

# performance    = my_testing_function(trained_model, testing_input, testing_target)

# noise_spectrum = estimate_noise(trained_model)

### Below is an example using tensorflow  ###

import tensorflow.keras as K

input_layer   = K.Input(shape=(None, axnum))

output_layer  = K.layers.LSTM(axnum2, return_sequences=False)(input_layer)

ml_model      = K.Model(input_layer, output_layer)

ml_model.compile(optimizer=K.optimizers.Adam(), loss='mse')

ml_model.fit(training_input, training_target, epochs=10, validation_data = (testing_input, testing_target))

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.3332 - val_loss: 0.3331
Epoch 2/10
1/1 [==============================] - 0s 237ms/step - loss: 0.3330 - val_loss: 0.3330
Epoch 3/10
1/1 [==============================] - 0s 256ms/step - loss: 0.3328 - val_loss: 0.3330
Epoch 4/10
1/1 [==============================] - 0s 254ms/step - loss: 0.3326 - val_loss: 0.3329
Epoch 5/10
1/1 [==============================] - 0s 247ms/step - loss: 0.3323 - val_loss: 0.3328
Epoch 6/10
1/1 [==============================] - 0s 271ms/step - loss: 0.3321 - val_loss: 0.3327
Epoch 7/10
1/1 [==============================] - 0s 259ms/step - loss: 0.3319 - val_loss: 0.3326
Epoch 8/10
1/1 [==============================] - 0s 258ms/step - loss: 0.3316 - val_loss: 0.3325
Epoch 9/10
1/1 [==============================] - 0s 230ms/step - loss: 0.3314 - val_loss: 0.3325
Epoch 10/10
1/1 [==============================] - 0s 279ms/step - loss: 0.3311 - val_loss: 0.3324


## Using the $V_O$ operators in a calculation

In [ ]:
'''Define a function to extract the data from the dataset.'''
def load_Vo_dataset(dataset_name, num_ex):
    
    # initialize empt lists for storing the data
    pulses         = []
    VX             = []
    VY             = []
    VZ             = []
    # 1) unzip the dataset zipfile
    fzip               = zipfile.ZipFile("%s.zip"%dataset_name, mode='r')
    
    # loop over example files
    #########################################################
    for idx_ex in range(num_ex):
    
        # 2) extract the example file from the dataset 
        fname = "%s_ex_%d"%(dataset_name, idx_ex)
        fzip.extract( fname )
    
        # 3) load the example file
        f     = open(fname,  "rb")
        data  = pickle.load(f)
    
        # 4) extract the useful information from the example file:
        
        # For noise spectroscopy, we need to extract a set of control pulse and the 
        # correpsonding informationally-complete observables
        
        # add the pair of input and target to the corresponding lists
        pulses.append( data["pulses"][0:1, :, 0, :])
        VX.append( data["Vo_operator"][0] )
        VY.append( data["Vo_operator"][1] )
        VZ.append( data["Vo_operator"][2] )
        
        # 5) close and delete the example file
        f.close()
        os.remove(fname)
    #########################################################
    # 5) close the dataset zipfile
    fzip.close()
    
    # 6) split the data into training and testing
    pulses            = np.concatenate(pulses, axis=0)
    VX                = np.concatenate(VX, axis=0)
    VY                = np.concatenate(VY, axis=0)
    VZ                = np.concatenate(VZ, axis=0)   
    
    return pulses, VX, VY, VZ

In [ ]:
# define the dataset parameters
dataset_name    = data2 #"G_1q_XY_XZ_N1N5_D" # dataset name
num_ex = 7                            # number of examples

In [ ]:
pulses, VX, VY, VZ = load_Vo_dataset(dataset_name, num_ex)

In [ ]:
print(pulses.shape, VX.shape, VY.shape, VZ.shape)

(7, 1024, 1) (7, 2, 2) (7, 2, 2) (7, 2, 2)


In [ ]:
'''Set input parameter shape based on number of qubits and set corresponding shape of flattened 
tensor for input into LSTM parameter.'''
# axnum = training_input.shape[2]
axnum3 = VX.shape[2]
lstmflat = axnum3 **2


2 4


In [ ]:
# use the Vo in a calculation

#############################################
##          PUT YOUR CODE HERE             ##

# result = my_calculation(pulses, VX)

In [ ]:
# train an ML model to learn Vo

#############################################
##          PUT YOUR CODE HERE             ##

# trained_model = my_training_function(training_input=pulses[0:100,;], training_target=VX[0:100, :])

# performance   = my_testing_function(trained_model, testing_input=pulses[100:,:], testing_target=Vx[100:,:])

### Below is an example using tensorflow  ###

import tensorflow.keras as K

input_layer   = K.Input(shape=(None, axnum))

output_layer  = K.layers.Reshape((axnum3,axnum3)) (  K.layers.LSTM(lstmflat, return_sequences=False)(input_layer) )

ml_model      = K.Model(input_layer, output_layer)

ml_model.compile(optimizer=K.optimizers.Adam(), loss='mse')

ml_model.fit(pulses, VX, epochs=10, validation_split=0.1)

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 0.4999 - val_loss: 0.4987
Epoch 2/10
1/1 [==============================] - 0s 221ms/step - loss: 0.4987 - val_loss: 0.4976
Epoch 3/10
1/1 [==============================] - 0s 213ms/step - loss: 0.4976 - val_loss: 0.4964
Epoch 4/10
1/1 [==============================] - 0s 207ms/step - loss: 0.4964 - val_loss: 0.4952
Epoch 5/10
1/1 [==============================] - 0s 241ms/step - loss: 0.4952 - val_loss: 0.4940
Epoch 6/10
1/1 [==============================] - 0s 229ms/step - loss: 0.4940 - val_loss: 0.4928
Epoch 7/10
1/1 [==============================] - 0s 178ms/step - loss: 0.4928 - val_loss: 0.4916
Epoch 8/10
1/1 [==============================] - 0s 229ms/step - loss: 0.4916 - val_loss: 0.4904
Epoch 9/10
1/1 [==============================] - 0s 197ms/step - loss: 0.4904 - val_loss: 0.4891
Epoch 10/10
1/1 [==============================] - 0s 240ms/step - loss: 0.4891 - val_loss: 0.4879


## Model the effect of control distortions

In [ ]:
# define the dataset parameters
dataset_name    = data2 #"G_1q_XY_XZ_N1N5_D" # dataset name
num_training_ex = 7                   # number of training examples
num_testing_ex  = 3                   # number of testing examples

In [ ]:
# load the dataset
training_input, training_target, testing_input, testing_target = load_dataset(dataset_name, num_training_ex, num_testing_ex)

In [ ]:
print(training_input.shape, training_target.shape, testing_input.shape, testing_target.shape)

(7, 1024, 1) (7, 18) (3, 1024, 1) (3, 18)


In [ ]:
'''Set shape parameters based on shape of inputs and outputs.'''
axnum = training_input.shape[2]
lstmout = training_target.shape[1]


(7, 1024, 1) (7, 18) (3, 1024, 1) (3, 18)
1


In [ ]:
# use the dataset for training and testing an ML algorithm

#############################################
##          PUT YOUR CODE HERE             ##

# trained_model = my_training_function(training_input, training_target)

# performance   = my_testing_function(trained_model, testing_input, testing_target)

### Below is an example using tensorflow  ###

import tensorflow.keras as K

input_layer   = K.Input(shape=(None, axnum))

output_layer  = K.layers.LSTM(lstmout, return_sequences=False)(input_layer)

ml_model      = K.Model(input_layer, output_layer)

ml_model.compile(optimizer=K.optimizers.Adam(), loss='mse')

ml_model.fit(training_input, training_target, epochs=10, validation_data = (testing_input, testing_target))

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.3332 - val_loss: 0.3332
Epoch 2/10
1/1 [==============================] - 0s 223ms/step - loss: 0.3328 - val_loss: 0.3332
Epoch 3/10
1/1 [==============================] - 0s 223ms/step - loss: 0.3324 - val_loss: 0.3332
Epoch 4/10
1/1 [==============================] - 0s 260ms/step - loss: 0.3319 - val_loss: 0.3332
Epoch 5/10
1/1 [==============================] - 0s 298ms/step - loss: 0.3315 - val_loss: 0.3332
Epoch 6/10
1/1 [==============================] - 0s 293ms/step - loss: 0.3310 - val_loss: 0.3333
Epoch 7/10
1/1 [==============================] - 0s 244ms/step - loss: 0.3305 - val_loss: 0.3333
Epoch 8/10
1/1 [==============================] - 0s 206ms/step - loss: 0.3300 - val_loss: 0.3333
Epoch 9/10
1/1 [==============================] - 0s 209ms/step - loss: 0.3294 - val_loss: 0.3334
Epoch 10/10
1/1 [==============================] - 0s 194ms/step - loss: 0.3288 - val_loss: 0.3335


## Learning a controller for a quantum system

In [ ]:
# define the dataset parameters
dataset_name    = data1 #"G_1q_XY_XZ_N1N5"   # dataset name
num_training_ex = 7                   # number of training examples
num_testing_ex  = 3                   # number of testing examples

In [ ]:
# load the dataset [inputs and targets are echanged for quantum control problem, this is the inverse of the 
# characterization problem]
training_target, training_input, testing_target, testing_input  = load_dataset(dataset_name, num_training_ex, num_testing_ex)

In [ ]:
print(training_input.shape, training_target.shape, testing_input.shape, testing_target.shape)



(7, 18) (7, 1024, 1) (3, 18) (3, 1024, 1)


In [ ]:
'''Set shape parameters based on shape of inputs and outputs.'''
axnum4 = training_input.shape[1]
axnum5 = training_target.shape[1]
axnum6 = training_target.shape[2]
axnum7 = axnum5 * axnum6

In [ ]:
# use the dataset for training and testing an ML algorithm

#############################################
##          PUT YOUR CODE HERE             ##

# trained_model = my_training_function(training_input, training_target)

# performance   = my_testing_function(trained_model, testing_input, testing_target)

### Below is an example using tensorflow  ###

import tensorflow.keras as K

input_layer   = K.Input(shape=(axnum4))

output_layer  = K.layers.Reshape((axnum5,axnum6))( K.layers.Dense(axnum7)(input_layer) )

ml_model      = K.Model(input_layer, output_layer)

ml_model.compile(optimizer=K.optimizers.Adam(), loss='mse')

ml_model.fit(training_input, training_target, epochs=10, validation_data = (testing_input, testing_target))

Epoch 1/10
1/1 [==============================] - 0s 221ms/step - loss: 439.2711 - val_loss: 523.0981
Epoch 2/10
1/1 [==============================] - 0s 12ms/step - loss: 439.1790 - val_loss: 523.1002
Epoch 3/10
1/1 [==============================] - 0s 12ms/step - loss: 439.0872 - val_loss: 523.1022
Epoch 4/10
1/1 [==============================] - 0s 12ms/step - loss: 438.9952 - val_loss: 523.1042
Epoch 5/10
1/1 [==============================] - 0s 12ms/step - loss: 438.9032 - val_loss: 523.1063
Epoch 6/10
1/1 [==============================] - 0s 12ms/step - loss: 438.8113 - val_loss: 523.1083
Epoch 7/10
1/1 [==============================] - 0s 12ms/step - loss: 438.7196 - val_loss: 523.1104
Epoch 8/10
1/1 [==============================] - 0s 12ms/step - loss: 438.6277 - val_loss: 523.1125
Epoch 9/10
1/1 [==============================] - 0s 12ms/step - loss: 438.5360 - val_loss: 523.1146
Epoch 10/10
1/1 [==============================] - 0s 12ms/step - loss: 438.4443 - val_los